### (WORK IN PROGRESS) 
# City of Mesa - Python API Use Cases

***
## Use Case #1 - Use Discovery API to get public dataset count

In [11]:
# Import libraries
import requests

In [6]:
# Define a re-usable method to return City of Mesa's public dataset count
def com_get_public_dataset_count():

    # Discovery API endpoint for domains
    endpoint = 'http://api.us.socrata.com/api/catalog/v1/domains'
    
    # City of Mesa domain on Socrata
    domain = 'data.mesaaz.gov'
    
    # Build the endpoint url for pulling datasets only
    url = endpoint + '?domains=' + domain + '&only=datasets'
    
    # Make the call to the Discovery endpoint 
    response = requests.get(url)
    
    # Parse the response for the count and return the value
    return response.json().get("results",[{}])[0].get("count","")


In [19]:
# Test the method!
print(com_get_public_dataset_count())

92


### Let's use Bokeh to chart the count
https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Python_Bokeh_Cheat_Sheet.pdf

In [10]:
# Bokeh imports
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

In [ ]:
# Tell Bokeh to output the chart to the notebook
output_notebook()

In [ ]:
# Set the x and y values where y is our count
x = ['Count']
y = [com_get_public_dataset_count()]
# Create a Bokeh figure including our count in the title
p = figure(x_range=x, plot_height=200, title='Dataset Count = ' + str(y))
# Set x and y into the figure as a vertical bar
p.vbar(x=x, top=y, width=0.5)
# Increase title size so we can see it
p.title.text_font_size = '12pt'
# Show the figure!
show(p)

***
## Use Case #2 - Use Metadata API to Copy Metadata Between Datasets

* The City of Mesa has 2 Socrata sites: Public and Internal
* We use the Metadata API to copy dataset Metadata between datasets on one site or between sites

### See Exercise #2 in this lab for sample code. 



***
## Use Case #3 - Use Discovery, Metadata, and SODA API to check for invalid dates

In [23]:
resource_endpoint = 'https://data.mesaaz.gov/resource/'
user_name = os.environ['MY_SOCRATA_USERNAME']
password = os.environ['MY_SOCRATA_PASSWORD']


In [24]:
# Discovery API to get list of public datasets (only look at first 5 for demo)
url = 'http://api.us.socrata.com/api/catalog/v1?domains=data.mesaaz.gov&only=datasets&limit=5'
response = requests.get(url)
results = response.json().get("results",[{}])
#print(results)

In [26]:
for dataset in results:
    dataset_resource = dataset['resource']
    dataset_id = dataset_resource.get('id')
    dataset_name = dataset_resource.get('name')
    column_fields = dataset_resource.get('columns_field_name')
    column_types = dataset_resource.get('columns_datatype')
    column_dict = dict(zip(column_fields, column_types))
    for k,v in column_dict.items():
        if v == 'calendar_date':
            # Using SoQL, get the min and max value of the date
            #min_url = resource_endpoint + dataset_id + '?$select=min(' + k + ')'
            #max_url = resource_endpoint + dataset_id + '?$select=max(' + k + ')'
            max_url = resource_endpoint + 'x78d-vqk4' + '?$select=max(year_month)'
            print (min_url)
            print (max_url)
            min_response = requests.get(min_url, auth=(user_name,password))
            max_response = requests.get(max_url, auth=(user_name,password))
            print(min_response.json())
            print(max_response.json())
            


https://data.mesaaz.gov/resource/24df-s3cy?$select=min(occurred_date)
https://data.mesaaz.gov/resource/x78d-vqk4?$select=max(year_month)
{'error': True, 'message': 'Internal error', 'status': 500}
[{'max_year_month': '2019-03-01T00:00:00.000'}]
https://data.mesaaz.gov/resource/24df-s3cy?$select=min(occurred_date)
https://data.mesaaz.gov/resource/x78d-vqk4?$select=max(year_month)
{'error': True, 'message': 'Internal error', 'status': 500}
[{'max_year_month': '2019-03-01T00:00:00.000'}]
https://data.mesaaz.gov/resource/24df-s3cy?$select=min(occurred_date)
https://data.mesaaz.gov/resource/x78d-vqk4?$select=max(year_month)
{'error': True, 'message': 'Internal error', 'status': 500}
[{'max_year_month': '2019-03-01T00:00:00.000'}]
https://data.mesaaz.gov/resource/24df-s3cy?$select=min(occurred_date)
https://data.mesaaz.gov/resource/x78d-vqk4?$select=max(year_month)
{'error': True, 'message': 'Internal error', 'status': 500}
[{'max_year_month': '2019-03-01T00:00:00.000'}]
https://data.mesaaz.

In [15]:
for dataset in results:
    dataset_id = dataset['resource'].get('id')
    dataset_url = 'https://data.mesaaz.gov/api/views/metadata/v1/' + dataset_id
    print(dataset_url)
    dataset_response = requests.get(dataset_url)
    dataset_results = dataset_response.json()
    print(dataset_results)
    

https://data.mesaaz.gov/api/views/metadata/v1/2gkz-7z4f
{'id': '2gkz-7z4f', 'name': 'Building Permits', 'attribution': 'Development Services', 'attributionLink': None, 'category': 'Permits & Licenses', 'createdAt': '2018-01-26T23:41:16+0000', 'dataUpdatedAt': '2019-03-20T16:04:32+0000', 'dataUri': 'https://data.mesaaz.gov/resource/2gkz-7z4f', 'description': 'Building permits issued by City of Mesa. Included: All building permits applied. Not Included: Applications for Documents or Services, Code Compliance Complaints, Planning Cases, or Engineering Projects. It is assumed that all permits have been issued based on meeting all building and zoning codes.', 'domain': 'data.mesaaz.gov', 'externalId': None, 'hideFromCatalog': False, 'hideFromDataJson': False, 'license': 'See Terms of Use', 'metadataUpdatedAt': '2019-02-20T21:46:02+0000', 'provenance': 'OFFICIAL', 'updatedAt': '2019-02-20T21:46:02+0000', 'webUri': 'https://data.mesaaz.gov/d/2gkz-7z4f', 'customFields': {'Terms of Use': {'Term